<a href="https://colab.research.google.com/github/ESA-PhiLab/WorldCrops/blob/hopfield/colab_notebooks/Hopfield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
gpu_id = 0

!wget https://raw.githubusercontent.com/ESA-PhiLab/WorldCrops/main/data/cropdata/Bavaria/sentinel-2/Training_bavaria.xlsx
!wget https://raw.githubusercontent.com/ESA-PhiLab/WorldCrops/main/data/cropdata/Bavaria/sentinel-2/Test_bavaria.xlsx
!wget https://raw.githubusercontent.com/ESA-PhiLab/WorldCrops/main/data/cropdata/Bavaria/Test_area.shp
!wget https://raw.githubusercontent.com/ESA-PhiLab/WorldCrops/main/data/cropdata/Bavaria/Test_area.shx

!wget https://raw.githubusercontent.com/ESA-PhiLab/WorldCrops/main/data/cropdata/Bavaria/sentinel-2/data2016-2018.xlsx
!wget https://raw.githubusercontent.com/ESA-PhiLab/WorldCrops/main/data/cropdata/Bavaria/sentinel-2/TestData.xlsx



In [ ]:
from modules import Hopfield, HopfieldPooling, HopfieldLayer

import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch import nn

bavaria_train = pd.read_excel(
    "Training_bavaria.xlsx")
bavaria_test = pd.read_excel(
    "Test_bavaria.xlsx")

bavaria_reordered = pd.read_excel(
    'data2016-2018.xlsx', index_col=0)
bavaria_test_reordered = pd.read_excel(
    'TestData.xlsx', index_col=0)

bavaria_test.drop(['Unnamed: 0'], axis=1, inplace=True)
bavaria_train.drop(['Unnamed: 0'], axis=1, inplace=True)

bavaria_train.Date = bavaria_train.Date.dt.strftime('%m-%d')

ModuleNotFoundError: ignored

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
bavaria_train.head()

,id,Date,area,NC,B1_mean,B2_mean,B3_mean,B4_mean,B5_mean,B6_mean,B7_mean,B8_mean,B8A_mean,B9_mean,B10_mean,B11_mean,B12_mean,NDVI_mean,NDRE_mean,REIP_mean
0,0,02-15,2.160921,115,1146.626562,831.977906,681.209049,436.75737,724.843314,2252.734038,3140.101011,3083.881444,3358.131258,1273.814711,8.160807,1151.194342,536.193014,0.752287,0.620349,727.821499
1,0,02-28,2.160921,115,1146.626562,831.977906,681.209049,436.75737,724.843314,2252.734038,3140.101011,3083.881444,3358.131258,1273.814711,8.160807,1151.194342,536.193014,0.752287,0.620349,727.821499
2,0,03-15,2.160921,115,1146.626562,831.977906,681.209049,436.75737,724.843314,2252.734038,3140.101011,3083.881444,3358.131258,1273.814711,8.160807,1151.194342,536.193014,0.752287,0.620349,727.821499
3,0,03-30,2.160921,115,1146.626562,831.977906,681.209049,436.75737,724.843314,2252.734038,3140.101011,3083.881444,3358.131258,1273.814711,8.160807,1151.194342,536.193014,0.752287,0.620349,727.821499
4,0,04-15,2.160921,115,1146.626562,831.977906,681.209049,436.75737,724.843314,2252.734038,3140.101011,3083.881444,3358.131258,1273.814711,8.160807,1151.194342,536.193014,0.752287,0.620349,727.821499


In [ ]:
#DataLoader
from torch.utils.data import Dataset
class data_set(Dataset):

    def __init__(self, inp, tar):
        self.inp = inp
        self.tar = tar

    def __len__(self):
        return self.inp.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self.inp[idx], self.tar[idx]


#Hopfield
class M1(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.hopfield = HopfieldLayer(
            scaling=1e12,
            input_bias=False,

            input_size=14,                           # R
            hidden_size=14,                          # W_K
            pattern_size=256,                        # W_V_1
            quantity=256,                            # W_K
            output_size=28,                           # W_V_2

            # do not project layer input
            state_pattern_as_static=True,
            stored_pattern_as_static=True,
            # pattern_projection_as_static=True,

            # # do not pre-process layer input
            # normalize_stored_pattern=False,
            # normalize_stored_pattern_affine=False,
            # normalize_state_pattern=False,
            # normalize_state_pattern_affine=False,
            # normalize_pattern_projection=False,
            # normalize_pattern_projection_affine=False,

            # # do not post-process layer output
            # disable_out_projection=True
            )
        self.Linear_m = nn.Linear(in_features=28, out_features=7)
        self.Linear_b = nn.Linear(in_features=bands, out_features=1)
        self.softmax = torch.nn.Softmax(dim=1)
    
    
    def forward(self, sample):
        x = self.hopfield(sample)
        y = self.Linear_m(x)
        y = torch.einsum('abc->acb', y)
        # y = torch.einsum('abc->acb', x)
        z = self.Linear_b(y)
        return self.softmax(z)[:,:,0]

In [ ]:
#%%
bavaria = bavaria_train.copy()

#adapt crop codex train
bavaria.loc[(bavaria.NC == 601)|(bavaria.NC == 602), 'NC'] = 600  #Kartoffeln jetzt alle Code 600
bavaria.loc[(bavaria.NC == 131)|(bavaria.NC == 476), 'NC'] = 131  #Wintergerste jetzt alle Code 131
bavaria.loc[(bavaria.NC == 411)|(bavaria.NC == 171)|(bavaria.NC == 410)|(bavaria.NC == 177), 'NC'] = 400  #Mais jetzt alle Code 400
bavaria.loc[(bavaria.NC == 311)|(bavaria.NC == 489), 'NC'] = 311  #Winterraps jetzt alle Code 311
#WW = 115
#ZR = 603
bavaria.loc[~((bavaria.NC == 600)|(bavaria.NC == 131)|(bavaria.NC == 400)|(bavaria.NC == 311)|(bavaria.NC == 115)|(bavaria.NC == 603)), 'NC'] = 1  

# #assign codes to test area
# bavaria_test.NC = bavaria_test.NC.astype(str).astype(int)
# bavaria_test.loc[(bavaria_test.NC == 410), 'NC'] = 400  #Corn
# bavaria_test.loc[~((bavaria_test.NC == 600)|(bavaria_test.NC == 131)|(bavaria_test.NC == 400)|(bavaria_test.NC == 311)|(bavaria_test.NC == 115)|(bavaria_test.NC == 603)), 'NC'] = 1  #rejection class other

#%%
codex = np.unique(bavaria.NC.values).tolist()
classes = [1,2,3,4,5,6,7]
codex2class = dict(zip(codex,classes))
class2codex = dict(zip(classes,classes))
bavaria['NC'] = bavaria['NC'].map(codex2class)

# codex = np.unique(bavaria_test.NC.values).tolist()
# classes = [1,2,3,4,5,6,7]
# codex2class = dict(zip(codex,classes))
# class2codex = dict(zip(classes,classes))
# bavaria_test['NC'] = bavaria_test['NC'].map(codex2class)



In [ ]:
bavaria.NC.unique()

array([2, 3, 7, 5, 4, 6, 1])

In [ ]:
entries = bavaria.shape[0]
crop_types = bavaria.NC.unique()
bands = 16

b = np.array(bavaria)
cnt = np.zeros((7))
data = {}
for n in range(7):
    data[n] = np.zeros((bands, 300, 14))
for m in range(7):
    cnt = 0
    fcnt = 0
    for n in range(entries):
        if(b[n,3]-1==m):
            # print(m)
            if (cnt==14):
                fcnt += 1
                cnt = 0
            data[m][:,fcnt,cnt] = b[n,4:] 
            cnt += 1

for n in range(7):
    for m in range(bands):
        for k in range(300):
            data[n][m,k,:] = data[n][m,k,:] / np.sqrt((np.einsum('a,a->',data[n][m,k], data[n][m,k])+0.001))

inp = torch.zeros((7,bands,300,14)).double()
target = torch.zeros((7,300))
for n in range(7):
    inp[n,:,:,0:14] = torch.tensor(data[n][:,:,0:14], dtype=float)
    target[n,:] = n
inp = torch.einsum('abcd->acbd', inp)

inp_a = torch.zeros((2100,bands,14))
target_a = torch.zeros((2100,7))
k = 0
for n in range(7):
    for m in range(300):
        inp_a[k,:,:] = inp[n,m]
        target_a[k,n] = 1
        k += 1



In [ ]:
set = data_set(inp_a, target_a)
loader = DataLoader(set, batch_size=1, shuffle=True)

model = M1()
# criterion = nn.MSELoss()
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-6) #, weight_decay=4e-5)


# sample = next(iter(loader))[0]
# target = next(iter(loader))[1]
# out = model(sample)
# print(out.shape)
# print(model(sample).shape)
# print(target.shape)
#%%
epochs = 500
for ep in range(epochs):
    print(ep)
    cor = 0
    for idx, data in enumerate(loader):
        
        sample = data[0]
        target = data[1]
        out = model(sample)
        if (torch.argmax(out[0]) == torch.argmax(target[0])):
            cor += 1

        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
    #     break

    if (ep%5==0):
        print('----', ep, '----')
        # print(out)
        # print(target)
        # print(torch.argmax(out[0]), torch.argmax(target[0]))
        print(cor, '/', idx)
    # break


0
---- 0 ----
489 / 2099
1
2
3
4
5
---- 5 ----
606 / 2099
6
7
8
9
10
---- 10 ----
682 / 2099
11
12
13
14
15
---- 15 ----
700 / 2099
16
17
18
19
20
---- 20 ----
835 / 2099
21
22
23
24
25
---- 25 ----
1019 / 2099
26
27
28
29
30
---- 30 ----
1041 / 2099
31
32
33
34
35
---- 35 ----
1076 / 2099
36
37
38
39
40
---- 40 ----
1105 / 2099
41
42
43
44
45
---- 45 ----
1092 / 2099
46
47
48
49
50
---- 50 ----
1120 / 2099
51
52
53
54
55
---- 55 ----
1139 / 2099
56
57
58
59
60
---- 60 ----
1144 / 2099
61
62
63
64
65
---- 65 ----
1181 / 2099
66
67
68
69
70
---- 70 ----
1177 / 2099
71
72
73
74
75
---- 75 ----
1177 / 2099
76
77
78
79
80
---- 80 ----
1176 / 2099
81
82
83
84
85
---- 85 ----
1179 / 2099
86
87
88
89
90
---- 90 ----
1179 / 2099
91
92
93
94
95
---- 95 ----
1172 / 2099
96
97
98
99
100
---- 100 ----
1194 / 2099
101
102
103
104
105
---- 105 ----
1202 / 2099
106
107
108
109
110
---- 110 ----
1201 / 2099
111
112
113
114
115
---- 115 ----
1213 / 2099
116
117
118
119
120
---- 120 ----
1220 / 2099
121